In [135]:
import torch
import torch.nn as nn
from torch.nn import functional as F
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
block_size = 8
batch_size = 4
max_iters = 10000
# eval_interval = 2500
learning_rate = 3e-4
eval_iters = 250


cuda


In [136]:
with open('wizard_of_oz.txt','r', encoding ='utf-8') as f:
    text = f.read()
chars = sorted(set(text))
print(chars)
print(len(chars))
vocab_size = len(chars)

['\n', ' ', '!', '"', '&', "'", '(', ')', '*', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\ufeff']
81


In [146]:
string_to_int = { ch:i for i,ch in enumerate(chars) }
int_to_string = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join(int_to_string[i] for i in l)

data = torch.tensor(encode(text), dtype = torch.long) # a new datatype FOR TORCH 
print(data[:100])

tensor([80,  1,  1,  1, 44, 62, 73, 65, 58, 22,  1, 28, 68, 71, 68, 73, 61, 78,
         1, 54, 67, 57,  1, 73, 61, 58,  1, 47, 62, 79, 54, 71, 57,  1, 62, 67,
         1, 39, 79,  0,  0, 25, 74, 73, 61, 68, 71, 22,  1, 36, 11,  1, 30, 71,
        54, 67, 64,  1, 26, 54, 74, 66,  0,  0, 33, 65, 65, 74, 72, 73, 71, 54,
        73, 68, 71, 22,  1, 34, 68, 61, 67,  1, 42, 11,  1, 38, 58, 62, 65, 65,
         0,  0, 36, 54, 67, 60, 74, 54, 60, 58])


In [147]:
n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    #print(ix)
    x = torch.stack([data[i:i+block_size]for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

x,y = get_batch('train')
print('inputs:')
#print(x.shape)
print(x)
print('target:')
print(y)

inputs:
tensor([[71, 58, 71, 72, 11,  0,  0,  3],
        [64, 58,  1, 68, 69, 58, 67, 10],
        [74, 71, 58, 64, 54,  1, 62, 67],
        [43, 68,  1, 76, 61, 58, 67,  1]], device='cuda:0')
target:
tensor([[58, 71, 72, 11,  0,  0,  3, 47],
        [58,  1, 68, 69, 58, 67, 10, 76],
        [71, 58, 64, 54,  1, 62, 67,  1],
        [68,  1, 76, 61, 58, 67,  1, 28]], device='cuda:0')


In [139]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X,Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [140]:


x = train_data[:block_size]
y = train_data[1:block_size+1]

for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print('when input is', context, 'target is', target)

when input is tensor([80]) target is tensor(1)
when input is tensor([80,  1]) target is tensor(1)
when input is tensor([80,  1,  1]) target is tensor(1)
when input is tensor([80,  1,  1,  1]) target is tensor(44)
when input is tensor([80,  1,  1,  1, 44]) target is tensor(62)
when input is tensor([80,  1,  1,  1, 44, 62]) target is tensor(73)
when input is tensor([80,  1,  1,  1, 44, 62, 73]) target is tensor(65)
when input is tensor([80,  1,  1,  1, 44, 62, 73, 65]) target is tensor(58)


In [141]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)  # This is going to be a learnable embedding
    
    def forward(self, index, targets=None):
        logits = self.token_embedding_table(index)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B * T, C)
            targets = targets.view(B * T)  # We reshaped this to match cross entropy of PyTorch
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, index, max_new_tokens):
        # Index is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # Get the predictions
            logits, _ = self.forward(index)
            # Focus only on the last time step
            logits = logits[:, -1, :]  # Becomes (B, C)
            # Apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1)  # (B, C)
            # Sample from the distribution
            index_next = torch.multinomial(probs, num_samples=1)  # (B, 1)
            # Append sampled index to the running sequence
            index = torch.cat((index, index_next), dim=1)
        return index

model = BigramLanguageModel(vocab_size)
m = model.to(device)

context = torch.zeros((1, 1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)



5utFTLp VIj)T5d3J
JIg4Mi0)JE﻿!.
Gy﻿!6y?hKy]UfX;dX1tFh2'j25jqERdcPeI!﻿m,]VIC-lJLo(
&lG*qYNW:UzR﻿ ViIcL_["*!S yyYm﻿lZeks2J,QDB;7I'FC,zYFn.av,N
7B?hawYripaTOD7Qryvl2'm?yxGgNpz,﻿"xq
c1!Sp0)3VoYo,MzW*CZZmMi0:F2kgc8R﻿Io:2'1Ju,EqRE*4 96 tLHKEKj5M;yMnU:qy﻿YA19Uj_*[)"37O4[4,b)cp0(1Ocjm&Lvn
)?qY'n prUm*YN8IQDCiCTr?Bhqb
M;2SMnc" Tn﻿*tBF,"]T1x(zOMj?ittdlCIqv﻿b4wgZBTj&XgjCIH6]MYA4s,M;!*yYKs*HZ,r7x7gI0"D
pr-]e!G3;,3TjCM'ipgyMrixaEwcsS-GfEXV(pTU;Gkgh5EzPxv,﻿FC28mLBU6y()kN.UQ6FBR9RTdCY4gEutNs]eanGfH3eJs_myROHhs


In [145]:
#create a pytorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)


for iter in range(max_iters):
    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f"step {iter}, train loss {losses['train']:.3f},val lass: {losses['val']:.3f}")
    # Sample a batch of data
    xb, yb = get_batch('train')
    # Evaluate the loss
    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True) #for context in gereral for rnn
    loss.backward()
    optimizer.step()

print("Final loss:", loss.item())


step 0, train loss 2.599,val lass: 2.684
step 250, train loss 2.616,val lass: 2.635
step 500, train loss 2.619,val lass: 2.675
step 750, train loss 2.619,val lass: 2.636
step 1000, train loss 2.590,val lass: 2.674
step 1250, train loss 2.590,val lass: 2.664
step 1500, train loss 2.585,val lass: 2.627
step 1750, train loss 2.584,val lass: 2.624
step 2000, train loss 2.583,val lass: 2.626
step 2250, train loss 2.574,val lass: 2.632
step 2500, train loss 2.592,val lass: 2.612
step 2750, train loss 2.598,val lass: 2.611
step 3000, train loss 2.567,val lass: 2.603
step 3250, train loss 2.562,val lass: 2.614
step 3500, train loss 2.575,val lass: 2.625
step 3750, train loss 2.572,val lass: 2.601
step 4000, train loss 2.575,val lass: 2.627
step 4250, train loss 2.548,val lass: 2.609
step 4500, train loss 2.558,val lass: 2.577
step 4750, train loss 2.520,val lass: 2.611
step 5000, train loss 2.544,val lass: 2.603
step 5250, train loss 2.549,val lass: 2.600
step 5500, train loss 2.534,val lass: 

In [133]:
context = torch.zeros((1,1), dtype = torch.long, device = device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


ndontop9!Zsta d aleal Mck busthed marnd al.&Ju
"Ww w  tgsway.

an wzlemiteylerinth
wan s s s ngre avplerovx&rp's rlplo wik"fe te
"ODD8;Mloheritluim "'Cofung]0xYBA, ou thew y buid, an,"apu t la?"
rl y,"
"
athe wingling " thin erowqsk he filla hPR91ure go hury, a t  k.3*uplatorernn ban35Z6m
LQuckI(ZWWBW;
t_s se  lince ber
Shq7J"
[80If aink singlldn
"
"fre oofikVY6QlkK5he dfeto bur, h69Wherld thece ckbby w]K1m 8MEpIflll Ph7J)v0ves
jj9
alok?"  Wmbingif, ind  sthein ag RY;pse nllZPq33RYsIpPho inu
:nd


# Optimizers
## important show it to ma'am

mean squared error= regression optimized line 

gradient descent
= minimizes loss function checks the efficient of output from the input 
momentum=  is extension of gradient descent smoothes training makes the curve go in same direction used in large neural net

rmsprop = i dont know

adam = very popular optimizes algorithm that combines the ideas of both the gradient descent and mse to adopt learning


adamw = modified adam that adds weight decay